In [3]:
import pandas as pd   
import seaborn as sns
# Load core libraries for data handling (pandas), plotting (matplotlib, seaborn), and numeric work (numpy).
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg') # Use TkAgg backend for better compatibility
import matplotlib.pyplot as plt 
plt.style.use('default')
# Force matplotlib to use the TkAgg backend so charts can display in a window environment.
import numpy as np
import warnings
warnings.filterwarnings('ignore') 
# Hide non-critical warning messages to keep output clean.
# Suppress warnings to keep output clean.


# FULL CLEANING PIPELINE (returns df AND monthly)

def clean_sales_data(file_path):    
    df = pd.read_csv("online_sales_data.csv")   #takes a file path and loads the CSV into a DataFrame
    print("Loading data...")  # Simple log to indicate the process started.

    print(df.head())  # shows first 5 rows
    print(df.info())  # shows column names, data types, non-null counts
    print(df.describe()) # shows summary statistics for numerical columns
    print(df.duplicated())  # True for duplicate rows
    print(df.duplicated().sum())  # count total duplicates
    print(df.isnull())  # True where missing
    print(df.isnull().sum())  # count missing values per column

    # 1. Fix data types

    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%d-%m-%Y %H:%M', errors='coerce')
    numeric_cols = ['Quantity', 'UnitPrice', 'CustomerID', 'Discount', 'ShippingCost']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Converts InvoiceDate from text to proper datetime using a day-month-year hour:minute format.
        # ​Converts selected columns to numeric, turning invalid values into NaN using errors='coerce'.
    
    product_to_cat = {
        "White Mug": "Home & Kitchen", "Headphones": "Electronics", "Desk Lamp": "Furniture",
        "USB Cable": "Electronics", "Office Chair": "Furniture", "Notebook": "Stationery",
        "Wireless Mouse": "Electronics", "Blue Pen": "Stationery", "Wall Clock": "Home Decor",
        "T-shirt": "Apparel", "Backpack": "Accessories", "Smartphone": "Electronics",
        "Water Bottle": "Accessories", "Sunglasses": "Accessories", "Coffee Maker": "Home & Kitchen"
    }
    #Dictionary mapping product names (e.g., "White Mug") to broader Category (e.g., "Home & Kitchen"), used to create clean product categories.

    
    # 2. Remove invalid records

    df = df.dropna(subset=['InvoiceNo', 'StockCode', 'Quantity', 'UnitPrice'])
    df = df[df['Quantity'] > 0]
    df = df[(df['UnitPrice'] > 0) & (df['UnitPrice'] <= 200)]
    df = df[df['Discount'] <= 1]

    # Drops rows with missing key fields.
    # Keeps only positive quantities.
    # Enforces unit price between 0 and 200 to remove outliers or errors.
    # Keeps discount values less than or equal to 1 (assuming discount is a fraction 0–1).
    
    # 3. Standardize text

    df['PaymentMethod'] = df['PaymentMethod'].str.strip().str.title()
    # Strips extra spaces and converts payment method text to Title Case (e.g., “credit card” → “Credit Card”) for consistency.

    # 4. Business calculations

    df['Stock'] = df['StockCode'].str.split('_').str[-1]
    #Extracts stock-related code from StockCode by splitting on _.
    
    df['Revenue'] = df['UnitPrice'] * (1 - df['Discount']) * df['Quantity']
    # Sales after discount: unit price × (1 − discount) × quantity.
    
    df['NetProfit'] = df['Revenue'] - df['ShippingCost']
    # Revenue minus shipping cost.
    
    df["Category"] = df["Description"].map(product_to_cat).fillna("Other")
    # Uses dictionary to classify product descriptions into categories, defaults to “Other”.
    
    df['TotalSales'] = df['Quantity'] * df['UnitPrice']
    # TotalSales:Gross sales before discount.
    
    df['NetSales'] = df['TotalSales'] * (1 - df['Discount'])
    # NetSales: Sales after discount.
    
    df['DiscountAmount'] = df['TotalSales'] * df['Discount']
    # DiscountAmount: Money value of discount.
    
    df['TotalPrice'] = df['NetSales'] + df['ShippingCost']  # Fixed calculation
    # TotalPrice: Final price customer pays, including shipping.
    
    # Extract Year/Month
    df["Year"] = pd.to_datetime(df["InvoiceDate"]).dt.year
    df["Month"] = pd.to_datetime(df["InvoiceDate"]).dt.month_name().str[:3]
    
    # Extracts Year and 3-letter month name from invoice date (e.g., “Jan”, “Feb”)
    # Create monthly summary (RETURN THIS TOO)

    monthly = (df.groupby(["Year", "Month"])["TotalPrice"]
               .sum()
               .reset_index()
               .sort_values(["Year", "Month"]))

    # Groups data by Year and Month.
    # Sums TotalPrice to get monthly revenue.
    # Resets index for a clean DataFrame and sorts by year and month for plotting.          
    
    # Additional calculations

    df['CustomerSegment'] = pd.qcut(df['TotalSales'], q=4, labels=['Bronze', 'Silver', 'Gold', 'Platinum'])
    df['Margin'] = ((df['NetSales'] - df['ShippingCost']) / df['NetSales'] * 100).round(1)
    df['DaysSince'] = (pd.Timestamp.now() - df['InvoiceDate']).dt.days
    df['LossFlag'] = df['NetProfit'].apply(lambda x: 'YES' if x < 0 else 'NO')
    
    # CustomerSegment: Divides customers into quartiles based on TotalSales (Bronze lowest to Platinum highest).
    # Margin: Profit margin % after shipping, rounded to 1 decimal.
    # DaysSince: Days since the order was placed (recency).
    # LossFlag: Flags orders where NetProfit is negative.

    # Round numerics
    for col in ['TotalSales', 'NetSales', 'DiscountAmount', 'TotalPrice', 'Revenue', 'NetProfit']:
        df[col] = df[col].round(1)
    
    return df, monthly  # Return both!
    
    # Rounds key monetary columns to one decimal for tidy reporting.
    # Returns both the cleaned row-level data (df) and the monthly summary.

# Execute cleaning (use clean df and monthly)
df, monthly = clean_sales_data('online_sales_data.csv')
df.to_csv('online_sales_clean.csv', index=False)
print(f"✅ Cleaning complete! {df.shape[0]} rows retained.")

# ALL PLOTS using clean df and monthly
plt.style.use('default')
# Reset plotting style to matplotlib’s default look

######### COUNTPLOT
plt.figure(figsize=(6,4))
sns.countplot(data=df, x="OrderPriority")
plt.title("Count of Orders by OrderPriority")
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

# Shows how many orders fall into each priority level (e.g., Low, Medium, High, Critical) as bars.
# ​Helps answer: “Do we mostly handle low‑priority orders, or is there a big volume of urgent/high‑priority ones?”
# If one bar is much taller, that priority dominates your workload.

######### SCATTER
plt.figure(figsize=(8, 6))
sns.scatterplot(data=df, x="UnitPrice", y="Quantity", hue="Category")
plt.title("Quantity vs UnitPrice by Category")
plt.tight_layout()
plt.show()

# Each dot is an order; x = UnitPrice, y = Quantity, color = Category.
# Shows whether cheaper products are sold in higher quantities, and if expensive ones are mostly single‑item orders.
# Also shows which categories tend to be bulk purchases vs occasional buys.

######### BAR PLOT
qty_by_cat = df.groupby("Category")["Quantity"].sum().reset_index()
plt.figure(figsize=(8,6))
sns.barplot(data=qty_by_cat, x="Category", y="Quantity")
plt.title("Total Quantity by Category")
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

# Bars show total units sold per Category.
# ​Directly answers: “Which product categories drive the most volume in the business?”
# High bars = strong volume categories; low bars = niche/low‑selling categories

######### LINE PLOT (NOW WORKS!)
plt.figure(figsize=(8,6))
sns.lineplot(data=monthly, x="Month", y="TotalPrice", hue="Year", marker="o")
plt.title("Monthly Revenue by Year")
plt.xlabel("Month")
plt.ylabel("TotalPrice")
plt.tight_layout()
plt.show()

# Uses your monthly summary to show TotalPrice per Month, with a different line for each Year.
# ​Answers: “Are there seasonal peaks (festivals, holidays)? Is this year better than last year in the same month?”
# Crossing lines show months where performance improved or declined year‑over‑year.

######### BOX PLOTS
plt.figure(figsize=(6, 4))
sns.boxplot(data=df, x="OrderPriority", y="TotalPrice")
plt.title("TotalPrice Distribution by Order Priority")
plt.tight_layout()
plt.show()

# For each OrderPriority, shows distribution of TotalPrice (median, typical range, and outliers).
# Answers: “Are high‑priority orders usually high‑value? Do low‑priority orders still bring large revenue sometimes?”
# Long boxes or many outliers indicate high variability in order values.

plt.figure(figsize=(8, 6))
sns.boxplot(data=df, x='Category', y='Quantity')
plt.title('Quantity Distribution by Category')
plt.tight_layout()
plt.show()

# Shows distribution of Quantity per order for each Category.
# Answers: “Which categories tend to be bulk buys vs single‑piece purchases?”
# Categories with higher medians or longer upper whiskers are typically ordered in larger quantities.

######### HISTOGRAMS
plt.figure(figsize=(8, 6))
sns.histplot(data=df, x="UnitPrice", hue="Category", bins=20, multiple="stack")
plt.title("Unit Price Distribution by Category")
plt.tight_layout()
plt.show()

# Shows how UnitPrice values are distributed and how each Category occupies the price range.
# ​Answers: “Which categories are low‑priced, mid‑priced, or premium?”
# You can see if a category is mostly low‑ticket or spread across many price points.

plt.figure(figsize=(8, 6))
sns.histplot(data=df[df['Quantity'] > 0], x='Quantity', bins=20, kde=True)
plt.title('Distribution of Quantity')
plt.tight_layout()
plt.show()

# Shows the distribution of Quantity per order, plus a smooth KDE curve.
# ​Answers: “What is the typical items per order? Are there many huge orders or mostly 1–2 quantity?”
# A right‑skewed shape means many small orders and a few very large ones.

######### KDE PLOTS
plt.figure(figsize=(8, 6))
sns.kdeplot(data=df, x="UnitPrice", fill=True)
plt.title("KDE of Unit Price")
plt.tight_layout()
plt.show()

# Smooth curve of where UnitPrice values are concentrated.
# ​Answers: “Around which prices do most products fall? Are there multiple price ‘clusters’?”
# Peaks in the curve show popular price points.

plt.figure(figsize=(8, 6))
sns.kdeplot(data=df[df['Quantity'] > 0], x="Quantity", fill=True)
plt.title("Quantity Distribution (KDE)")
plt.tight_layout()
plt.show()

# Similar to the histogram but smoother; shows where most Quantities lie.
# ​Answers: “Is the typical order 1–3 units, or higher?”
# Helps quickly see if large quantities are rare or common. 

plt.figure(figsize=(8,4))
sns.kdeplot(data=df, x='TotalSales', hue='Month', fill=True)
plt.title("Monthly Density by Totalsales Status")
plt.xlabel("Monthly TotalSales (INR)")
plt.tight_layout()
plt.show()

# For each Month, shows a density curve of TotalSales per order.
# ​Answers: “Do some months have generally higher order values than others?”
# If curves for certain months shift right, those months typically have higher order values.

######### VIOLIN PLOTS
plt.figure(figsize=(8, 6))
sns.violinplot(data=df, x="Category", y="Quantity", inner="quartile")
plt.title('Quantity Distribution by Category (Violin Plot)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Each “violin” shows distribution of Quantity per Category, with inner lines for quartiles.
# ​Answers: “How does order size distribution differ between categories, not just averages?”
# Wide sections of the violin show where many orders are concentrated.

plt.figure(figsize=(6, 4))
sns.violinplot(data=df, x="OrderPriority", y="TotalPrice")  # Changed FinalPrice -> TotalPrice
plt.title("TotalPrice Distribution by Order Priority")
plt.tight_layout()
plt.show()

# Shows detailed distribution of TotalPrice for each priority level.
# ​Answers: “Do urgent orders have a broader range or systematically higher values than low priority ones?”
# You can see if high‑priority is mostly high‑value or mixed.

######### SWARM PLOTS
plt.figure(figsize=(8, 6))
sns.swarmplot(data=df, x="Category", y="Quantity", size=2)
plt.title("Quantity per Order by Category (Swarm Plot)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Each dot is a single order, jittered so they don’t overlap.
# ​Answers: “How many actual orders fall at different quantities for each category?”
# Good for seeing clusters (e.g., many points at Quantity = 1–2).

plt.figure(figsize=(6, 4))
sns.swarmplot(data=df, x="OrderPriority", y="TotalPrice", size=3)  # Fixed column
plt.title("TotalPrice per Order by Priority")
plt.tight_layout()
plt.show()

# Shows every order’s TotalPrice per priority, as dots.
# ​Answers: “Are high‑priority orders mostly high‑value? Do low‑priority orders sometimes spike very high?”
# Patterns of dots quickly show concentration vs rare extreme values.

######### PIE CHARTS
country_counts = df['Country'].value_counts().head(10)
plt.figure(figsize=(8, 6))
colors = sns.color_palette('pastel', len(country_counts))
plt.pie(country_counts.values, labels=country_counts.index, colors=colors, autopct='%.1f%%')
plt.title('Sales Distribution by Country (Top 10)')
plt.tight_layout()
plt.show()

# Shows each of the top 10 Countries’ share of number of orders.
# ​Answers: “Which countries contribute most to order count?”
# Good for understanding geographic spread at a high level.

payment_counts = df['PaymentMethod'].value_counts()
plt.figure(figsize=(8, 6))
colors = sns.color_palette('tab10', len(payment_counts))
plt.pie(payment_counts.values, labels=payment_counts.index, colors=colors, autopct='%.1f%%')
plt.title('Payment Method Distribution')
plt.tight_layout()
plt.show()

# Shows proportion of orders paid by each PaymentMethod (e.g., Credit Card, UPI, COD).
# ​Answers: “Which payment channels are most popular?”
# Useful for payment strategy and risk analysis.

# Country profit pie
country_profit = df.groupby('Country')['NetProfit'].sum().nlargest(8).round(0)
plt.figure(figsize=(8, 6))
colors = sns.color_palette('husl', len(country_profit))
plt.pie(country_profit.values, labels=country_profit.index, colors=colors, autopct='%.1f%%')
plt.title('Profit Distribution by Country (Top 8)')
plt.tight_layout()
plt.show()

####### Heat map
pivot = df.pivot_table(
    index="Country",
    columns="Category",
    values="TotalPrice",
    aggfunc="sum"
)

plt.figure(figsize=(12, 8))
sns.heatmap(pivot, cmap="YlGnBu")
plt.title("Heatmap – TotalPrice by Country and Category")
plt.xlabel("Category")
plt.ylabel("Country")
plt.tight_layout()
plt.show()

#  pair plot

cols = ["Quantity", "UnitPrice", "TotalPrice", "NetProfit"]
plt.figure(figsize=(8, 8))
sns.pairplot(df[cols], diag_kind="kde")
plt.suptitle("Pairplot – Quantity, UnitPrice, TotalPrice, NetProfit", y=1.02)
plt.show()


# Shows share of total NetProfit contributed by the top 8 countries.
# Answers: “Which countries are most profitable, not just high volume?”
# A country can be low‑volume but high‑profit if margins are better

print("✅ All  plots completed! Check online_sales_data.csv for your cleaned data.")
print ("✅ Project complete!", "Weldone!") 
#Confirms that all plots are generated and the project is complete.
df                                                      
#displays the cleaned DataFrame in an interactive environment.

Loading data...
   InvoiceNo StockCode Description  Quantity       InvoiceDate  UnitPrice  \
0     100005  SKU_1312    Notebook        45  12-01-2022 19:00      89.78   
1     100015  SKU_1084   USB Cable         2  05-11-2021 16:00      38.49   
2     100037  SKU_1678    Notebook        42  05-06-2021 09:00      28.38   
3     100056  SKU_1860  Wall Clock         3  01-07-2023 17:00      27.32   
4     100059  SKU_1058   Desk Lamp        24  21-10-2024 14:00      17.62   

   CustomerID      Country  Discount  PaymentMethod  ShippingCost  \
0     88722.0       Norway      0.16    Credit Card         26.38   
1     73709.0      Germany      0.18        paypall         19.34   
2     55846.0  Netherlands      0.15        paypall         23.83   
3     20877.0        Italy      0.19        paypall         16.49   
4     18268.0        Italy      0.16  Bank Transfer         23.42   

      Category SalesChannel  ReturnStatus ShipmentProvider WarehouseLocation  \
0  Electronics       Onlin

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Discount,PaymentMethod,...,TotalSales,NetSales,DiscountAmount,TotalPrice,Year,Month,CustomerSegment,Margin,DaysSince,LossFlag
0,100005,SKU_1312,Notebook,45,2022-01-12 19:00:00,89.78,88722.0,Norway,0.16,Credit Card,...,4040.1,3393.7,646.4,3420.1,2022,Jan,Platinum,99.2,1444,NO
1,100015,SKU_1084,USB Cable,2,2021-11-05 16:00:00,38.49,73709.0,Germany,0.18,Paypall,...,77.0,63.1,13.9,82.5,2021,Nov,Bronze,69.4,1512,NO
2,100037,SKU_1678,Notebook,42,2021-06-05 09:00:00,28.38,55846.0,Netherlands,0.15,Paypall,...,1192.0,1013.2,178.8,1037.0,2021,Jun,Gold,97.6,1665,NO
3,100056,SKU_1860,Wall Clock,3,2023-07-01 17:00:00,27.32,20877.0,Italy,0.19,Paypall,...,82.0,66.4,15.6,82.9,2023,Jul,Bronze,75.2,909,NO
4,100059,SKU_1058,Desk Lamp,24,2024-10-21 14:00:00,17.62,18268.0,Italy,0.16,Bank Transfer,...,422.9,355.2,67.7,378.6,2024,Oct,Silver,93.4,431,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,117572,SKU_1801,White Mug,30,2025-07-15 00:00:00,1.41,57119.0,Netherlands,0.14,Paypall,...,42.3,36.4,5.9,41.6,2025,Jul,Bronze,85.6,164,NO
996,117580,SKU_1322,Wall Clock,32,2022-07-25 20:00:00,87.75,51274.0,Spain,0.33,Paypall,...,2808.0,1881.4,926.6,1899.6,2022,Jul,Platinum,99.0,1250,NO
997,117621,SKU_1238,T-shirt,43,2025-01-07 02:00:00,48.72,64209.0,United States,0.05,Paypall,...,2095.0,1990.2,104.7,2002.8,2025,Jan,Platinum,99.4,353,NO
998,117633,SKU_1937,White Mug,5,2020-04-21 12:00:00,63.66,86969.0,Norway,0.43,Paypall,...,318.3,181.4,136.9,200.7,2020,Apr,Bronze,89.4,2075,NO
